
# **Frequent Itemsets with PySpark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system.

Follow the steps to install the dependencies:

In [144]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [145]:
!wget -qN https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

In [146]:
!pip install -q findspark

Set the location of Java and Spark by running the following code:

In [147]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

Install PySpark and run a local spark session to test the installation:

In [148]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [149]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

Let's create a spark DataFrame to confirm that we can run PySpark, and preload that DataFrame with test baskets.

| Transaction ID | Stock Items |
|:--------------:|:---------------- |
| 100 | milk, coke, beer |
| 200 | milk, pepsi, juice |
| 300 | milk, beer |
| 400 | coke, juice |
| 500 | milk, pepsi, beer |
| 600 | milk, coke, beer, juice |
| 700 | coke, beer, juice |
| 800 | beer, coke |

Each DataFrame row is `<Transaction ID, [Stock Items]>`

In [150]:
import numpy as np

In [151]:
m,c,b,p,j = 12,3,2,15,9
basket_df = spark.createDataFrame([
    (100, [m,c,b]),
    (200, [m,p,j]),
    (300, [m,b]),
    (400, [c,j]),
    (500, [m,p,b]),
    (600, [m,c,b,j]),
    (700, [c,b,j]),
    (800, [b,c])
], ["id", "items"])
basket_df.show()
stockIDs = {b: 'Beer', c: 'Coke', m: 'Milk', j: 'Juice', p: 'Pepsi'}

+---+-------------+
| id|        items|
+---+-------------+
|100|   [12, 3, 2]|
|200|  [12, 15, 9]|
|300|      [12, 2]|
|400|       [3, 9]|
|500|  [12, 15, 2]|
|600|[12, 3, 2, 9]|
|700|    [3, 2, 9]|
|800|       [2, 3]|
+---+-------------+



<hr>

# PySpark Code

## FP-Growth Algorithm

References:

* PySpark [introduction for FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth).
* [PySpark Dataframes](https://sparkbyexamples.com/pyspark/convert-pandas-to-pyspark-dataframe/)

First, run FP-Growth example from the documentation

In [152]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.5, minConfidence=0.6)
model = fpGrowth.fit(basket_df)

# Display generated association rules.
model.associationRules.show()

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+----------+----------+------------------+------------------+-------+
|antecedent|consequent|        confidence|              lift|support|
+----------+----------+------------------+------------------+-------+
|       [3]|       [2]|               0.8|1.0666666666666667|    0.5|
|      [12]|       [2]|               0.8|1.0666666666666667|    0.5|
|       [2]|       [3]|0.6666666666666666|1.0666666666666667|    0.5|
|       [2]|      [12]|0.6666666666666666|1.0666666666666667|    0.5|
+----------+----------+------------------+------------------+-------+



## 1. Interpreting association rules

The above table, has columns antecedent, consequent, confidence, lift and support.


#### Association Rules with changed minSupport and minConfidence values

Modify the support threshold to be 0.375 and minimum confidence to be 0.75.

In [153]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="items", minSupport=0.375, minConfidence=0.75)
model = fpGrowth.fit(basket_df)

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

+-------+----+
|  items|freq|
+-------+----+
|    [3]|   5|
| [3, 2]|   4|
|    [2]|   6|
|   [12]|   5|
|[12, 2]|   4|
|    [9]|   4|
| [9, 3]|   3|
+-------+----+

+----------+----------+----------+------------------+-------+
|antecedent|consequent|confidence|              lift|support|
+----------+----------+----------+------------------+-------+
|       [3]|       [2]|       0.8|1.0666666666666667|    0.5|
|      [12]|       [2]|       0.8|1.0666666666666667|    0.5|
|       [9]|       [3]|      0.75|               1.2|  0.375|
+----------+----------+----------+------------------+-------+



## 2. New association rules
The third row of the result shows a low support (0.375) and a high lift (1.2).


## 3. Association Rules for an Online Retail Dataset

Here we process a sampled dataset from a UK-based online retailer. We'll be working with a 8050 record subset.

* Read in the data from the dataset `online_retail_III.csv`.
* There are a couple of wrinkles to keep in mind:
    * An invoice represents a shopping cart and it can contain multiple items.
    * Some invoice numbers start with a "C." Invoice number C123456 is to be interpreted as a return of items in invoice 123456. The `inum` column represents the Invoice number as well as the credit (return). In other words, Invoice numbers `123456` and `C123456` would have `inum` == 123456.

In [154]:
import pandas as pd

df_orig = pd.read_csv('https://storage.googleapis.com/singhj-public-data/online_retail_II.csv')
df_orig.dropna(inplace=True)
df_orig.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### Data Scrubbing

Remove the rows we should filter away. They aren't necessarily visible in the summary view but we know they exist.

* StockCode `POST`,
* StockCode `M`.


In [155]:
df = df_orig[(df_orig["StockCode"]!="POST") & (df_orig["StockCode"]!="M")]

## 4. Connecting Online Retail Data to FP-Growth

Adapt the DataFrame to look like `df_basket` above.
* `df_orig` is a Pandas DataFrame whereas `df_basket`-equivalent will have to be Spark DataFrames.
* `Invoice` and `StockCode` are strings but FP-Growth needs inputs to be integers. You'd need to map strings to integers before feeding them to FP-Growth and convert the resulting antecedents and consequents back.

### Establish the mapping between Invoice IDs, StockCodes and unique integers.

In [156]:
# Create function for mapping lists of stockCodes.
def stock_code_to_index(list):
  idx_list = []
  for i in list:
    idx_list.append(item_idx[i])

  return idx_list

def stock_index_to_code(list):
  idx_list = []
  for i in list:
    idx_list.append(item_codes[i])

  return idx_list

# inv_baskets stores multiple items (stock codes) in one basket (invoice)
inv_baskets = df.groupby("Invoice")
inv_baskets = inv_baskets["StockCode"].agg(lambda x: list(set(x))).reset_index(name='StockCode')

# Conversely, create each item and list all baskets it appears in
item = df.groupby("StockCode")
item = item["Invoice"].agg(lambda x: list(set(x))).reset_index(name='Invoice')

# Map each item and each basket to their unique index codes
inv_key = inv_baskets["Invoice"]
inv_value = range(len(inv_key))
inv_idx = dict(zip(inv_key, inv_value))

item_key = item["StockCode"]
item_value = range(len(item_key))
item_idx = dict(zip(item_key, item_value))
item_codes = dict(zip(item_value, item_key))

# Convert table of strings into table of integer codes
inv_baskets["Invoice"] = inv_baskets["Invoice"].map(lambda x: inv_idx[x])
inv_baskets["StockCode"] = inv_baskets["StockCode"].map(lambda x: stock_code_to_index(x))

invoice_df = spark.createDataFrame(inv_baskets)

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


## 5. Fine-tuning FP-Growth runs

* Set `minConfidence` = 0.75.
* Set `minSupport` such that the total number of association rules is between 10 and 20. (If `minSupport` is small, the number of association rules will increase. As it increases, the number of association rules will decrease.).



In [157]:
fpGrowth_2 = FPGrowth(itemsCol="StockCode", minSupport=0.01, minConfidence=0.75)
model_2 = fpGrowth_2.fit(invoice_df)

# Display generated association rules.
model_2.associationRules.show()

/content/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+------------+----------+------------------+------------------+--------------------+
|  antecedent|consequent|        confidence|              lift|             support|
+------------+----------+------------------+------------------+--------------------+
|       [534]|     [532]|0.8023255813953488| 47.69139879182447|0.010936261745726254|
|      [1879]|    [1881]|0.7709163346613546|30.103907975524958| 0.01752518962979735|
|[1879, 1617]|    [1881]|0.8330241187384044|32.529186741009404|0.010166421374391487|
|      [1880]|    [1879]|0.8154613466334164|  35.8713649144072|0.014808105966262877|
|      [1880]|    [1881]|0.7793017456359103|30.431354196295295|0.014151477414242046|
|      [3385]|    [3384]|0.7544097693351425| 35.63476733977173|0.012589154307709726|
|[1880, 1879]|    [1881]|0.8440366972477065|32.959222576432325|0.012498584852258576|
|[1880, 1881]|    [1879]|            0.8832| 38.85112350597609|0.012498584852258576|
|[4073, 1586]|    [4071]|0.7538940809968847| 9.959836101473948|0.

## 6. Final Association Rules

Present the resulting Association Rules in terms of the original StockCodes and Descriptions, in descending order of lift.

In [158]:
rules_by_lift = model_2.associationRules.orderBy("lift", ascending=False)
rules_by_lift.show()

+------------+----------+------------------+------------------+--------------------+
|  antecedent|consequent|        confidence|              lift|             support|
+------------+----------+------------------+------------------+--------------------+
|      [1929]|    [1926]|0.7641357027463651| 60.37218839319001|0.010709838107098382|
|      [1926]|    [1929]|0.8461538461538461| 60.37218839319001|0.010709838107098382|
|       [504]|     [511]|            0.7872| 49.88047058823529| 0.01114004302049134|
|       [534]|     [532]|0.8023255813953488| 47.69139879182447|0.010936261745726254|
|[1880, 1881]|    [1879]|            0.8832| 38.85112350597609|0.012498584852258576|
|      [1880]|    [1879]|0.8154613466334164|  35.8713649144072|0.014808105966262877|
|      [3385]|    [3384]|0.7544097693351425| 35.63476733977173|0.012589154307709726|
|[1880, 1879]|    [1881]|0.8440366972477065|32.959222576432325|0.012498584852258576|
|[1879, 1617]|    [1881]|0.8330241187384044|32.529186741009404|0.

In [159]:
# Convert integer indexes back to original codes
rules_pd = rules_by_lift.toPandas()
rules_pd["antecedent"] = rules_pd["antecedent"].map(lambda x: stock_index_to_code(x))
rules_pd["consequent"] = rules_pd["consequent"].map(lambda x: stock_index_to_code(x))
rules_pd

,antecedent,consequent,confidence,lift,support
0,[22748],[22745],0.764136,60.372188,0.010710
1,[22745],[22748],0.846154,60.372188,0.010710
2,[21086],[21094],0.787200,49.880471,0.011140
3,[21124],[21122],0.802326,47.691399,0.010936
4,"[22698, 22699]",[22697],0.883200,38.851124,0.012499
5,[22698],[22697],0.815461,35.871365,0.014808
6,[82581],[82580],0.754410,35.634767,0.012589
7,"[22698, 22697]",[22699],0.844037,32.959223,0.012499
8,"[22697, 22423]",[22699],0.833024,32.529187,0.010166
9,[22698],[22699],0.779302,30.431354,0.014151


In [160]:
df_stock_descr = df[["StockCode", "Description"]].drop_duplicates(subset=['StockCode'])
stock_list = df_stock_descr.StockCode.tolist()
descr_list = df_stock_descr.Description.tolist()
stock_descr = dict(zip(stock_list, descr_list))

def stock_code_to_description(list):
  descriptions = []
  for i in list:
    descriptions.append(stock_descr[str(i)])

  return descriptions

In [161]:
rules_pd["antecedent_description"] = rules_pd["antecedent"].map(lambda x: stock_code_to_description(x))
rules_pd["consequent_description"] = rules_pd["consequent"].map(lambda x: stock_code_to_description(x))
rules_pd


,antecedent,consequent,confidence,lift,support,antecedent_description,consequent_description
0,[22748],[22745],0.764136,60.372188,0.010710,[POPPY'S PLAYHOUSE KITCHEN],[POPPY'S PLAYHOUSE BEDROOM ]
1,[22745],[22748],0.846154,60.372188,0.010710,[POPPY'S PLAYHOUSE BEDROOM ],[POPPY'S PLAYHOUSE KITCHEN]
2,[21086],[21094],0.787200,49.880471,0.011140,[SET/6 RED SPOTTY PAPER CUPS],[SET/6 RED SPOTTY PAPER PLATES]
3,[21124],[21122],0.802326,47.691399,0.010936,[SET/10 BLUE SPOTTY PARTY CANDLES],[SET/10 PINK SPOTTY PARTY CANDLES]
4,"[22698, 22699]",[22697],0.883200,38.851124,0.012499,"[PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",[GREEN REGENCY TEACUP AND SAUCER]
5,[22698],[22697],0.815461,35.871365,0.014808,[PINK REGENCY TEACUP AND SAUCER],[GREEN REGENCY TEACUP AND SAUCER]
6,[82581],[82580],0.754410,35.634767,0.012589,[TOILET METAL SIGN],[BATHROOM METAL SIGN]
7,"[22698, 22697]",[22699],0.844037,32.959223,0.012499,"[PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",[ROSES REGENCY TEACUP AND SAUCER ]
8,"[22697, 22423]",[22699],0.833024,32.529187,0.010166,"[GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",[ROSES REGENCY TEACUP AND SAUCER ]
9,[22698],[22699],0.779302,30.431354,0.014151,[PINK REGENCY TEACUP AND SAUCER],[ROSES REGENCY TEACUP AND SAUCER ]
